In [262]:
# Dependencies and Setup
import pandas as pd

school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete_rename = school_data_complete.rename(columns={"student_name":"Student Name", "gender":"Gender", "grade":"Grade", "school_name":"School", "reading_score":"Reading Score", "math_score":"Math Score", "budget":"Budget", "type":"Type"})

school_data_complete_rename.head()

,Student ID,Student Name,Gender,Grade,School,Reading Score,Math Score,School ID,Type,size,Budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [263]:
#District Summary

#Total Schools
schools_unique = school_data_complete_rename["School"].unique()
total_schools = len(schools_unique)

#Total Students
total_students = len(school_data_complete_rename["Student ID"])

#Total Budget
budgets = school_data_complete_rename["Budget"].unique()
total_budget = budgets.sum()

#Average Math Score
average_math_score = school_data_complete_rename["Math Score"].mean()

#Average Reading Score
average_reading_score = school_data_complete_rename["Reading Score"].mean()

#Overall Pass Rate (Overall Average Score)
overall_pass_rate = float((average_reading_score + average_math_score)) / 2

#Percent students passing math
pass_total_m = 0
for score in school_data_complete_rename["Math Score"]:
    if score >= 70:
        pass_total_m += 1
percent_pass_m = float((pass_total_m) / len(school_data_complete_rename["Math Score"])) * 100

#Percent students passing reading
pass_total_r = 0
for score in school_data_complete_rename["Reading Score"]:
    if score >= 70:
        pass_total_r += 1
percent_pass_r = float((pass_total_r) / len(school_data_complete_rename["Reading Score"])) * 100

#Put data in a dataframe
data_summary_df = pd.DataFrame({"Total Number Schools":[total_schools], "Total Number Students":[total_students], "Average Reading Score":[average_reading_score], "Reading Pass Rate":[percent_pass_r], "Average Math Score":[average_math_score], "Math Pass Rate":[percent_pass_m], "Overall Pass Rate":[overall_pass_rate], "Total Budget":[total_budget]})

#Formatting Data
data_summary_df["Total Number Students"] = data_summary_df["Total Number Students"].map("{:,}".format)
data_summary_df["Average Reading Score"] = data_summary_df["Average Reading Score"].map("{:.2f}".format)
data_summary_df["Reading Pass Rate"] = data_summary_df["Reading Pass Rate"].map("{:.2f}%".format)
data_summary_df["Average Math Score"] = data_summary_df["Average Math Score"].map("{:.2f}".format)
data_summary_df["Math Pass Rate"] = data_summary_df["Math Pass Rate"].map("{:.2f}%".format)
data_summary_df["Overall Pass Rate"] = data_summary_df["Overall Pass Rate"].map("{:.2f}%".format)
data_summary_df["Total Budget"] = data_summary_df["Total Budget"].map("${:,.2f}".format)
data_summary_df.head()

,Total Number Schools,Total Number Students,Average Reading Score,Reading Pass Rate,Average Math Score,Math Pass Rate,Overall Pass Rate,Total Budget
0,15,"39,170",81.88,85.81%,78.99,74.98%,80.43%,"$24,649,428.00"


In [264]:
#School Summary

#Group data by school
data_by_school = school_data_complete_rename.groupby("School")

#School type
school_type = data_by_school["Type"].unique()
data_by_school_df = pd.DataFrame({"Type":school_type})
data_by_school_df.loc[["Bailey High School", "Figueroa High School", "Ford High School", "Hernandez High School", "Huang High School", "Johnson High School", "Rodriguez High School"], "Type"] = "District"
data_by_school_df.loc[["Cabrera High School", "Griffin High School", "Holden High School", "Pena High School", "Shelton High School", "Thomas High School", "Wilson High School", "Wright High School"], "Type"] = "Charter"

#Number of Students per School
data_by_school_df["Number of Students"] = data_by_school["Student ID"].count()

#School Budget
data_by_school_df["Budget"] = budgets
data_by_school_df["Budget"] = data_by_school_df["Budget"]

#Budget Per Student
data_by_school_df["Budget Per Student"] = (data_by_school_df["Budget"] / data_by_school_df["Number of Students"]).map("${:,.2f}".format)

#Formating Number of Students and Budget
data_by_school_df["Number of Students"] = data_by_school_df["Number of Students"].map("{:,}".format)
data_by_school_df["Budget"] = data_by_school_df["Budget"].map("${:,.2f}".format)

#Average Math Score
data_by_school_df["Average Math Score"] = (data_by_school["Math Score"].sum() / data_by_school["Math Score"].count()).map("{:.2f}".format)

#Average Reading Score
data_by_school_df["Average Reading Score"] = (data_by_school["Reading Score"].sum() / data_by_school["Reading Score"].count()).map("{:.2f}".format)

#Math Pass Rate
data_by_school_df["Math Percent Pass"] = ((data_by_school["Math Score"].apply(lambda x: (x>=70).sum()) / data_by_school["Math Score"].count()) * 100)

#Reading Pass Rate
data_by_school_df["Reading Percent Pass"] = ((data_by_school["Reading Score"].apply(lambda x: (x>=70).sum()) / data_by_school["Reading Score"].count()) * 100)

#Overall Pass Rate
data_by_school_df["Overall Pass Rate"] = ((data_by_school_df["Math Percent Pass"] + data_by_school_df["Reading Percent Pass"]) / 2).map("{:.2f}%".format)
data_by_school_df["Math Percent Pass"] = data_by_school_df["Math Percent Pass"].map("{:.2f}%".format)
data_by_school_df["Reading Percent Pass"] = data_by_school_df["Reading Percent Pass"].map("{:.2f}%".format)
data_by_school_df.head()

,Type,Number of Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,Math Percent Pass,Reading Percent Pass,Overall Pass Rate
School,,,,,,,,,
Bailey High School,District,"4,976","$1,910,635.00",$383.97,77.05,81.03,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,884,411.00","$1,014.21",83.06,83.98,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,056,600.00",$358.29,76.71,81.16,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$3,022,020.00","$1,103.33",77.10,80.75,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%


In [265]:
#Top Performing Schools
top_schools = data_by_school_df.sort_values("Overall Pass Rate", ascending=False)
top_schools.head()

,Type,Number of Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,Math Percent Pass,Reading Percent Pass,Overall Pass Rate
School,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,884,411.00","$1,014.21",83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$3,094,650.00","$1,892.75",83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,Charter,"2,283","$1,763,916.00",$772.63,83.27,83.99,93.87%,96.54%,95.20%


In [266]:
#Bottom Performing Schools
top_schools.tail()

,Type,Number of Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,Math Percent Pass,Reading Percent Pass,Overall Pass Rate
School,,,,,,,,,
Ford High School,District,"2,739","$3,022,020.00","$1,103.33",77.10,80.75,68.31%,79.30%,73.80%
Johnson High School,District,"4,761","$248,087.00",$52.11,77.07,80.97,66.06%,81.22%,73.64%
Huang High School,District,"2,917","$3,124,928.00","$1,071.28",76.63,81.18,65.68%,81.32%,73.50%
Figueroa High School,District,"2,949","$1,056,600.00",$358.29,76.71,81.16,65.99%,80.74%,73.36%
Rodriguez High School,District,"3,999","$1,049,400.00",$262.42,76.84,80.74,66.37%,80.22%,73.29%


In [267]:
#Math Scores By Grade
data_by_grades = school_data_complete_rename.groupby(["School", "Grade"])
avg_mscore_by_grade = data_by_grades["Math Score"].mean()

#9th grade
ninth_grade_m = avg_mscore_by_grade.loc(axis=0)[:, ["9th"], :].values
data_by_school_df["9th"] = ninth_grade_m
data_by_school_df["9th"] = data_by_school_df["9th"].map("{:.2f}".format)

#10th grade
tenth_grade_m = avg_mscore_by_grade.loc(axis=0)[:, ["10th"], :].values
data_by_school_df["10th"] = tenth_grade_m
data_by_school_df["10th"] = data_by_school_df["10th"].map("{:.2f}".format)

#11th grade
eleventh_grade_m = avg_mscore_by_grade.loc(axis=0)[:, ["11th"], :].values
data_by_school_df["11th"] = eleventh_grade_m
data_by_school_df["11th"] = data_by_school_df["11th"].map("{:.2f}".format)

#12th grade
twelfth_grade_m = avg_mscore_by_grade.loc(axis=0)[:, ["12th"], :].values
data_by_school_df["12th"] = twelfth_grade_m
data_by_school_df["12th"] = data_by_school_df["12th"].map("{:.2f}".format)

#Set the dataframe
data_by_grades_df = data_by_school_df[["9th", "10th", "11th", "12th"]]
data_by_grades_df.head()

,9th,10th,11th,12th
School,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36


In [268]:
# Reading Scores by Grade
avg_rscore_by_grade = data_by_grades["Reading Score"].mean()

#9th grade
ninth_grade_r = avg_rscore_by_grade.loc(axis=0)[:, ["9th"], :].values
data_by_school_df["9th"] = ninth_grade_r
data_by_school_df["9th"] = data_by_school_df["9th"].map("{:.2f}".format)

#10th grade
tenth_grade_r = avg_rscore_by_grade.loc(axis=0)[:, ["10th"], :].values
data_by_school_df["10th"] = tenth_grade_r
data_by_school_df["10th"] = data_by_school_df["10th"].map("{:.2f}".format)

#11th grade
eleventh_grade_r = avg_rscore_by_grade.loc(axis=0)[:, ["11th"], :].values
data_by_school_df["11th"] = eleventh_grade_r
data_by_school_df["11th"] = data_by_school_df["11th"].map("{:.2f}".format)

#12th grade
twelfth_grade_r = avg_rscore_by_grade.loc(axis=0)[:, ["12th"], :].values
data_by_school_df["12th"] = twelfth_grade_r
data_by_school_df["12th"] = data_by_school_df["12th"].map("{:.2f}".format)

#Set the dataframe
data_by_grades_df = data_by_school_df[["9th", "10th", "11th", "12th"]]
data_by_grades_df.head()

,9th,10th,11th,12th
School,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01


In [269]:
#Scores by School Spending
bud_per_stu = data_by_school_df["Budget Per Student"].replace({"\$":"", ",":""}, regex=True)
per_stu_df = pd.DataFrame({"Budget Per Student": bud_per_stu})
per_stu_df["Budget Per Student"] = pd.to_numeric(per_stu_df["Budget Per Student"])

#Set bins
spending_bins = [0, 700, 1400, 2100, 2800]
spending_labels = ["<$700", "$700-$1,400", "1,400-$2,100", "$2,100-2,800"]

#Add bins to school grouped data
per_stu_df["Spending Range (By Student)"] = pd.cut(per_stu_df["Budget Per Student"], spending_bins, labels=spending_labels)

#Add other data to dataframe
per_stu_df["Average Math Score"] = data_by_school_df["Average Math Score"]
per_stu_df["Math Percent Pass"] = data_by_school_df["Math Percent Pass"]
per_stu_df["Average Reading Score"] = data_by_school_df["Average Reading Score"]
per_stu_df["Reading Percent Pass"] = data_by_school_df["Reading Percent Pass"]
per_stu_df["Overall Pass Rate"] = data_by_school_df["Overall Pass Rate"]

#Resetting the index & dtypes
per_stu_df = per_stu_df.reset_index()
per_stu_df["Average Math Score"] = pd.to_numeric(per_stu_df["Average Math Score"])
per_stu_df["Average Reading Score"] = pd.to_numeric(per_stu_df["Average Reading Score"])
per_stu_df["Math Percent Pass"] = per_stu_df["Math Percent Pass"].replace({"\%":""}, regex=True)
per_stu_df["Reading Percent Pass"] = per_stu_df["Reading Percent Pass"].replace({"\%":""}, regex=True)
per_stu_df["Math Percent Pass"] = pd.to_numeric(per_stu_df["Math Percent Pass"])
per_stu_df["Reading Percent Pass"] = pd.to_numeric(per_stu_df["Reading Percent Pass"])
per_stu_df["Overall Pass Rate"] = per_stu_df["Overall Pass Rate"].replace({"\%":""}, regex=True)
per_stu_df["Overall Pass Rate"] = pd.to_numeric(per_stu_df["Overall Pass Rate"])

#Regroup data by bins
per_stu_df_grouped = per_stu_df.groupby(["Spending Range (By Student)"])

#Average the scores by bin
avg_mscore_bin = per_stu_df_grouped["Average Math Score"].mean().map("{:.2f}".format)
avg_rscore_bin = per_stu_df_grouped["Average Reading Score"].mean().map("{:.2f}".format)
per_stu_df_grouped_df = pd.DataFrame({"Average Math Score":avg_mscore_bin, "Average Reading Score":avg_rscore_bin})
avg_mpass_bin = per_stu_df_grouped["Math Percent Pass"].mean().map("{:.2f}%".format)
avg_rpass_bin = per_stu_df_grouped["Reading Percent Pass"].mean().map("{:.2f}%".format)
avg_opass_bin = per_stu_df_grouped["Overall Pass Rate"].mean().map("{:.2f}%".format)
per_stu_df_grouped_df = pd.DataFrame({"Average Math Score":avg_mscore_bin, "Average Reading Score":avg_rscore_bin, "Average Math Percent Pass":avg_mpass_bin, "Average Reading Percent Pass":avg_rpass_bin, "Average Overall Pass Rate":avg_opass_bin})
per_stu_df_grouped_df

,Average Math Score,Average Reading Score,Average Math Percent Pass,Average Reading Percent Pass,Average Overall Pass Rate
Spending Range (By Student),,,,,
<$700,79.48,82.08,76.65%,86.83%,81.74%
"$700-$1,400",80.02,82.48,80.50%,88.55%,84.52%
"1,400-$2,100",83.39,83.79,93.57%,96.58%,95.08%
"$2,100-2,800",83.80,83.81,92.51%,96.25%,94.38%


In [270]:
#Scores by school size
stu_per_school = data_by_school_df["Number of Students"].replace({",":""}, regex=True)
per_school_df = pd.DataFrame({"Number of Students": stu_per_school})
per_school_df["Number of Students"] = pd.to_numeric(per_school_df["Number of Students"])

#Set bins
size_bins = [0, 1000, 3000, 5000]
size_labels = ["Small", "Medium", "Large"]

#Add bins to school grouped data
per_school_df["School Size"] = pd.cut(per_school_df["Number of Students"], size_bins, labels=size_labels)

#Add the rest of the data
per_school_df["Average Math Score"] = data_by_school_df["Average Math Score"]
per_school_df["Math Percent Pass"] = data_by_school_df["Math Percent Pass"]
per_school_df["Average Reading Score"] = data_by_school_df["Average Reading Score"]
per_school_df["Reading Percent Pass"] = data_by_school_df["Reading Percent Pass"]
per_school_df["Overall Pass Rate"] = data_by_school_df["Overall Pass Rate"]

#Resetting the index & dtypes
per_school_df = per_school_df.reset_index()
per_school_df["Average Math Score"] = pd.to_numeric(per_school_df["Average Math Score"])
per_school_df["Average Reading Score"] = pd.to_numeric(per_school_df["Average Reading Score"])
per_school_df["Math Percent Pass"] = per_school_df["Math Percent Pass"].replace({"\%":""}, regex=True)
per_school_df["Reading Percent Pass"] = per_school_df["Reading Percent Pass"].replace({"\%":""}, regex=True)
per_school_df["Math Percent Pass"] = pd.to_numeric(per_school_df["Math Percent Pass"])
per_school_df["Reading Percent Pass"] = pd.to_numeric(per_school_df["Reading Percent Pass"])
per_school_df["Overall Pass Rate"] = per_school_df["Overall Pass Rate"].replace({"\%":""}, regex=True)
per_school_df["Overall Pass Rate"] = pd.to_numeric(per_school_df["Overall Pass Rate"])

#Regroup data by bins
per_school_df_grouped = per_school_df.groupby(["School Size"])

#Average the scores by bin
avg_mscore_sizebin = per_school_df_grouped["Average Math Score"].mean().map("{:.2f}".format)
avg_rscore_sizebin = per_school_df_grouped["Average Reading Score"].mean().map("{:.2f}".format)
per_stu_df_grouped_df = pd.DataFrame({"Average Math Score":avg_mscore_sizebin, "Average Reading Score":avg_rscore_sizebin})
avg_mpass_sizebin = per_school_df_grouped["Math Percent Pass"].mean().map("{:.2f}%".format)
avg_rpass_sizebin = per_school_df_grouped["Reading Percent Pass"].mean().map("{:.2f}%".format)
avg_opass_sizebin = per_school_df_grouped["Overall Pass Rate"].mean().map("{:.2f}%".format)
per_school_df_grouped_df = pd.DataFrame({"Average Math Score":avg_mscore_sizebin, "Average Reading Score":avg_rscore_sizebin, "Average Math Percent Pass":avg_mpass_sizebin, "Average Reading Percent Pass":avg_rpass_sizebin, "Average Overall Pass Rate":avg_opass_sizebin})
per_school_df_grouped_df

,Average Math Score,Average Reading Score,Average Math Percent Pass,Average Reading Percent Pass,Average Overall Pass Rate
School Size,,,,,
Small,83.82,83.93,93.55%,96.10%,94.82%
Medium,81.18,82.93,84.65%,91.32%,87.98%
Large,77.06,80.92,66.47%,81.06%,73.76%


In [302]:
#Scores by School Tye

#Setting up data
school_by_type_df = data_by_school_df.reset_index()
school_by_type_df["Average Math Score"] = pd.to_numeric(school_by_type_df["Average Math Score"])
school_by_type_df["Average Reading Score"] = pd.to_numeric(school_by_type_df["Average Reading Score"])
school_by_type_df["Math Percent Pass"] = school_by_type_df["Math Percent Pass"].replace({"\%":""}, regex=True)
school_by_type_df["Reading Percent Pass"] = school_by_type_df["Reading Percent Pass"].replace({"\%":""}, regex=True)
school_by_type_df["Overall Pass Rate"] = school_by_type_df["Overall Pass Rate"].replace({"\%":""}, regex=True)
school_by_type_df["Math Percent Pass"] = pd.to_numeric(school_by_type_df["Math Percent Pass"])
school_by_type_df["Reading Percent Pass"] = pd.to_numeric(school_by_type_df["Reading Percent Pass"])
school_by_type_df["Overall Pass Rate"] = pd.to_numeric(school_by_type_df["Overall Pass Rate"])
school_by_type_df = school_by_type_df.drop(["School", "Number of Students", "Budget", "Budget Per Student", "9th", "10th", "11th", "12th"], axis=1)

#Group schools by type
school_by_type_grouped = school_by_type_df.groupby("Type")

#Find averages
avg_mscore_type = school_by_type_grouped["Average Math Score"].mean().map("{:.2f}".format)
avg_rscore_type = school_by_type_grouped["Average Reading Score"].mean().map("{:.2f}".format)
avg_mpass_type = school_by_type_grouped["Math Percent Pass"].mean().map("{:.2f}%".format)
avg_rpass_type = school_by_type_grouped["Reading Percent Pass"].mean().map("{:.2f}%".format)
avg_opass_type = school_by_type_grouped["Overall Pass Rate"].mean().map("{:.2f}%".format)
school_by_type_grouped_df = pd.DataFrame({"Average Math Score":avg_mscore_type, "Average Reading Score":avg_rscore_type, "Average Math Percent Pass":avg_mpass_type, "Average Reading Percent Pass":avg_rpass_type, "Average Overall Pass Rate":avg_opass_type})
school_by_type_grouped_df

,Average Math Score,Average Reading Score,Average Math Percent Pass,Average Reading Percent Pass,Average Overall Pass Rate
Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,95.10%
District,76.96,80.97,66.55%,80.80%,73.67%
